In [2]:
from osgeo import gdal
import sys
import os
import pandas as pd

In [3]:
#path1 = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/aligned/id_397615_ortho.tif'
#path2 = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/aligned/id_397615_oh.tif'
#path_out = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/aligned/gdal_out_al.tif'
#path_out2 = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/aligned/gdal_out_al_oh.tif'

In [4]:
path1 = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/tiles/ortho/tile_ortho_1.tif'
path2 = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/tiles/dsm/tile_dsm_1.tif'

path_out = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/tiles/dsm_02/tile_dsm_1.tif'

In [4]:
ds1 = gdal.Open(path1)
ds2 = gdal.Open(path2)

## align to 1m

In [5]:
print("Driver: {}/{}".format(ds2.GetDriver().ShortName,
                            ds2.GetDriver().LongName))
print()
print("Size is {} x {} x {}".format(ds2.RasterXSize,
                                    ds2.RasterYSize,
                                    ds2.RasterCount))
print()
print("Projection is {}".format(ds2.GetProjection()))
print()
geotransform = ds2.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: GTiff/GeoTIFF

Size is 103 x 103 x 1

Projection is PROJCS["MGI / Austria Lambert",GEOGCS["MGI",DATUM["Militar_Geographische_Institute",SPHEROID["Bessel 1841",6377397.155,299.1528128000033,AUTHORITY["EPSG","7004"]],TOWGS84[577.326,90.129,463.919,5.137,1.474,5.297,2.4232],AUTHORITY["EPSG","6312"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4312"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",49],PARAMETER["standard_parallel_2",46],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.3333333333333],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","31287"]]

Origin = (373852.8101525671, 389129.64519568067)
Pixel Size = (0.9999999999999799, -0.99999999999998)


In [6]:
# aligning to 1m
#gdal.Warp(path_out, ds1, targetAlignedPixels = True, xRes = geotransform[1], yRes = geotransform[1]) 

## align to 0.2m

In [7]:
print("Driver: {}/{}".format(ds1.GetDriver().ShortName,
                            ds1.GetDriver().LongName))
print()
print("Size is {} x {} x {}".format(ds1.RasterXSize,
                                    ds1.RasterYSize,
                                    ds1.RasterCount))
print()
print("Projection is {}".format(ds1.GetProjection()))
print()
geotransform = ds1.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: GTiff/GeoTIFF

Size is 515 x 515 x 4

Projection is PROJCS["Lambert Conformal Conic",GEOGCS["wkt_datum_1",DATUM["wkt_datum_1",SPHEROID["Bessel",6377397.155,299.152813106079]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",49],PARAMETER["standard_parallel_2",46],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.3333333333333],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]

Origin = (373852.97009303217, 389129.6834124083)
Pixel Size = (0.2, -0.2)


In [8]:
# aligning to 0.2m
geotransform = ds1.GetGeoTransform()
gdal.Warp(path_out, ds2, 
          targetAlignedPixels = True, 
          xRes = geotransform[1], 
          yRes = geotransform[1], 
          width = ds1.RasterXSize, 
          height = ds1.RasterXSize) 

In [9]:
geotransform

(373852.97009303217, 0.2, 0.0, 389129.6834124083, 0.0, -0.2)

In [ ]:
###############################################################################
# Test -tr

def test_gdalwarp_7():
    if test_cli_utilities.get_gdalwarp_path() is None:
        return 'skip'

    gdaltest.runexternal(test_cli_utilities.get_gdalwarp_path() + ' -tr 120 120 tmp/testgdalwarp_gcp.tif tmp/testgdalwarp7.tif')

    ds = gdal.Open('tmp/testgdalwarp7.tif')
    if ds is None:
        return 'fail'

    expected_gt = (440720.0, 120.0, 0.0, 3751320.0, 0.0, -120.0)
    if not gdaltest.geotransform_equals(expected_gt, ds.GetGeoTransform(), 1e-9) :
        gdaltest.post_reason('Bad geotransform')
        return 'fail'

    ds = None

    return 'success'

###############################################################################

In [ ]:
###############################################################################
# Test -tap

def test_gdalwarp_32():
    if test_cli_utilities.get_gdalwarp_path() is None:
        return 'skip'

    (out, err) = gdaltest.runexternal_out_and_err(test_cli_utilities.get_gdalwarp_path() + ' -tap ../gcore/data/byte.tif tmp/testgdalwarp32.tif',
                                                  check_memleak = False)
    if err.find('-tap option cannot be used without using -tr') == -1:
        gdaltest.post_reason('expected error')
        return 'fail'

    gdaltest.runexternal(test_cli_utilities.get_gdalwarp_path() + ' -tr 100 50 -tap ../gcore/data/byte.tif tmp/testgdalwarp32.tif')

    ds = gdal.Open('tmp/testgdalwarp32.tif')
    if ds is None:
        return 'fail'

    expected_gt = (440700.0, 100.0, 0.0, 3751350.0, 0.0, -50.0)
    got_gt = ds.GetGeoTransform()
    if not gdaltest.geotransform_equals(expected_gt, got_gt, 1e-9) :
        gdaltest.post_reason('Bad geotransform')
        print(got_gt)
        return 'fail'

    if ds.RasterXSize != 13 or ds.RasterYSize != 25:
        gdaltest.post_reason('Wrong raster dimensions : %d x %d' % (ds.RasterXSize, ds.RasterYSize) )
        return 'fail'

    ds = None

    return 'success'

###############################################################################

In [5]:
####
# is working
####
bashCommand = "gdalwarp -tr 0.2 0.2 -tap " + path2 + " " + path_out
# execute bash command
os.system(bashCommand)

0